In [2]:
def upper_right_2(N,q,y):
    from scipy.linalg import circulant
    def alpha(N,q,y):
        import math
        L = []
        k = int((N-1)/2)
        #print(k)
        for j in range(k):
            L.append(j-k)
        for j in range(k,N-2+1):
            L.append(j+1-k)
        s = math.floor(N*q**(1/y)) + 1
        L.append(s)
        return L
    L = alpha(N,q,y)
    return circulant(L).T

def upper_right(N,q,y,a):
    from scipy.linalg import circulant
    return circulant(a).T

def upper_left(N):
    from scipy.sparse import identity
    return identity(N).toarray().astype(int)

def bottom_left(N):
    import numpy as np
    return np.zeros((N,N)).astype(int)

def bottom_right(N,q):
    from scipy.sparse import identity
    return q*identity(N).toarray().astype(int)

def ntru_matrix(N,q,y,a):
    import numpy as np
    return np.block([
    [upper_left(N), upper_right(N,q,y,a)],

    [bottom_left(N), bottom_right(N,q)]])

def mat2fp(A):
    from fpylll import IntegerMatrix
    L = IntegerMatrix(A.shape[0],A.shape[1])
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            L[i,j] = int(A[i,j])
    return L

# converts from type fpylll.integetMatrix to numpy.array
def fp2mat(A):
    L = np.zeros(shape=(A.nrows, A.ncols))
    for i in range(A.nrows):
        for j in range(A.ncols):
            L[i, j] = int(A[i, j])
            L = np.array(L).astype(int)
    return L


def lllreduce(A):
    from fpylll.fplll.lll import LLLReduction,LLL
    from fpylll import FPLLL
    _ = FPLLL.set_precision(53)
    A_fp = mat2fp(A)
    print(A_fp.nrows,"X",A_fp.ncols)
    LLL.reduction(A_fp, delta =0.99 )
    print("Is reduced?",LLL.is_reduced(A_fp))
    return A_fp

In [3]:
from numpy import linalg as LA
import numpy as np
import math
from termcolor import colored # if you don't have termcolor, comment this line.
from fpylll.fplll.lll import LLLReduction,LLL
from fpylll import SVP
import random
for N,q,y in [[21,2**5,1.5],[22,2**5,2],[25,2**9,2], [27,2**9,2]]:
    print(N,q,y)
    aN_ = [random.randint(-2,2) for i in range(N-1)] + [math.floor(N*q**(1/y))]
    aN = [aN_[i] for i in range(N)]
    random.shuffle(aN)
    A = ntru_matrix(N,q,y,aN)
    A_fp = lllreduce(A)
    LLL.is_reduced(A_fp)
    _=SVP.shortest_vector(A_fp)
    # the next line calls SVP function
    shortest = A_fp[0].norm()
    right_part = q**(1/y)
    # according the hypothesis of the theorem λ_1>q^(1/y)
    #print("1st:",left_part>shortest)
    if shortest>right_part:
        print(colored("True",'red'))
    else:
        print(colored("False",'blue'))
    print("\n" )

21 32 1.5
42 X 42
Is reduced? True
True


22 32 2
44 X 44
Is reduced? True
True


25 512 2
50 X 50
Is reduced? True
True


27 512 2
54 X 54
Is reduced? True
True


